While there are a number of "classic" machine learning datasets (IRIS, Boston, Digits) in recent time there are some more recent 'Standard' datasets that everyone is at least somewhat familliar with. 
Thank to Kaggle, the word titanic now means something different to data scientists than to james cameron fans. 

Let's go through this, to show an end to end workflow, without using the timesaving technique of pipelines:


# EDA:
Let's look at the options for eda. 
Basic eda shows us that women outlive men, The children survived a lot, and every indicator social class was valuable. Here is a link to my eda. This will focus on taking the dataset and extracting features that are useful. 


In [1]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm, linear_model, datasets
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler,PolynomialFeatures, LabelEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from scipy import stats
%matplotlib inline
titanic = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')

/home/nerd/Data/Conda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
titanic['Cabin'] =titanic['Cabin'].fillna(value='?')
titanic['Embarked'] =titanic['Embarked'].fillna(value='C')
titanic['Age'] =titanic['Age'].fillna(value='?')
testdf['Cabin'] =testdf['Cabin'].fillna(value='?')
testdf['Embarked'] =testdf['Embarked'].fillna(value='C')
testdf['Age'] =testdf['Age'].fillna(value='?')

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,?,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,?,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,?,S


In [4]:
testdf.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,?,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7.0000,?,S
2,894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,?,Q
3,895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,?,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,?,S


So now we know what the dataset looks like. A few things jump out: Fare represents not only what social class people were in, but the duplicate values likely indicate tickets sold for the same room. 
SibSP and Parent children(as per our eda) have a few interesting points, namely, being part of a very large family (8 or more) gaurantedd your death in the training dating, and the single people (no parents, siblings or spouses), especially men, didn't do so well. 

In [5]:
# let's begin to feature engineer. 
titanic['FamilyCount']= (titanic['SibSp'])+(titanic['Parch'])
titanic['IsFemale'] = (titanic.Sex=='female').astype(int)
titanic['IsMale'] = (titanic.Sex=='male').astype(int)
testdf['FamilyCount']= (testdf['SibSp'])+(testdf['Parch'])
testdf['IsFemale'] = (testdf.Sex=='female').astype(int)
testdf['IsMale'] = (testdf.Sex=='male').astype(int)


In [6]:
# people's titles are valuable, and what deck they were on affects their survival. 
def titlepull(title_string):
    title_list = title_string.split()
    for possible_title in title_list:
        if possible_title[-1] == '.':
            return possible_title
def cabinpull(cabin_string):
    if cabin_string[0]=='?':
        return '?'
    else:
        return cabin_string[0]
titanic['Deck']=titanic.Cabin.apply(cabinpull)
titanic['Title']= titanic.Name.apply(titlepull)
testdf['Deck']=testdf.Cabin.apply(cabinpull)
testdf['Title']= testdf.Name.apply(titlepull)
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilyCount,IsFemale,IsMale,Deck,Title
886,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.00,?,S,0,0,1,?,Rev.
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S,0,1,0,B,Miss.
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,?,1,2,W./C. 6607,23.45,?,S,3,1,0,?,Miss.
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C,0,0,1,C,Mr.
890,891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,?,Q,0,0,1,?,Mr.


It's time to make some dummies. 
Dummies are one of the best way to use categorical variables. By making a new set of columns for each possible category, we have numerical values (0 or 1) for everyone. 


In [7]:
embarked_dummies= pd.get_dummies(titanic.Embarked, prefix='Embarked_')
title_dummies= pd.get_dummies(titanic.Title,prefix='Title')
parc_dummies=  pd.get_dummies(titanic.Parch,prefix='parch')
pclass_dummies=   pd.get_dummies(titanic.Pclass,prefix='plcass')
sibsp_dummies=  pd.get_dummies(titanic.SibSp,prefix='sibsp')
family_dummies = pd.get_dummies(titanic.FamilyCount,prefix='family')
deck_dummies = pd.get_dummies(titanic.Deck,prefix='Deck')

test_embarked_dummies= pd.get_dummies(testdf.Embarked, prefix='Embarked_')
test_title_dummies= pd.get_dummies(testdf.Title,prefix='Title')
test_parc_dummies=  pd.get_dummies(testdf.Parch,prefix='parch')
test_pclass_dummies=   pd.get_dummies(testdf.Pclass,prefix='plcass')
test_sibsp_dummies=  pd.get_dummies(testdf.SibSp,prefix='sibsp')
testfamily_dummies = pd.get_dummies(testdf.FamilyCount,prefix='family')
testdeck_dummies = pd.get_dummies(testdf.Deck,prefix='Deck')



In [8]:
dummieslist = [title_dummies,embarked_dummies,titanic, parc_dummies,pclass_dummies,sibsp_dummies,family_dummies,deck_dummies]
titanic_feature_engineered= pd.concat(dummieslist, axis=1)
testdummieslist = [test_title_dummies,test_embarked_dummies,testdf, test_parc_dummies,test_pclass_dummies,test_sibsp_dummies,testfamily_dummies,testdeck_dummies]
test_feature_engineered= pd.concat(testdummieslist, axis=1)

In [9]:
test_feature_engineered.tail()

,Title_Col.,Title_Dona.,Title_Dr.,Title_Master.,Title_Miss.,Title_Mr.,Title_Mrs.,Title_Ms.,Title_Rev.,Embarked__C,...,family_7,family_10,Deck_?,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
413,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
414,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
415,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
416,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
417,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [10]:
titanic_feature_engineered.tail()

,Title_Capt.,Title_Col.,Title_Countess.,Title_Don.,Title_Dr.,Title_Jonkheer.,Title_Lady.,Title_Major.,Title_Master.,Title_Miss.,...,family_10,Deck_?,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
886,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
887,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
888,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
890,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0



Now I want to impute (or make up) some missing data. Specifically I want to make up the missing age data. 

In [11]:
titanic[titanic['Age']=='?']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilyCount,IsFemale,IsMale,Deck,Title
5,6,0,3,"Moran, Mr. James",male,?,0,0,330877,8.4583,?,Q,0,0,1,?,Mr.
17,18,1,2,"Williams, Mr. Charles Eugene",male,?,0,0,244373,13.0000,?,S,0,0,1,?,Mr.
19,20,1,3,"Masselmani, Mrs. Fatima",female,?,0,0,2649,7.2250,?,C,0,1,0,?,Mrs.
26,27,0,3,"Emir, Mr. Farred Chehab",male,?,0,0,2631,7.2250,?,C,0,0,1,?,Mr.
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,?,0,0,330959,7.8792,?,Q,0,1,0,?,Miss.
29,30,0,3,"Todoroff, Mr. Lalio",male,?,0,0,349216,7.8958,?,S,0,0,1,?,Mr.
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,?,1,0,PC 17569,146.5208,B78,C,1,1,0,B,Mrs.
32,33,1,3,"Glynn, Miss. Mary Agatha",female,?,0,0,335677,7.7500,?,Q,0,1,0,?,Miss.
36,37,1,3,"Mamee, Mr. Hanna",male,?,0,0,2677,7.2292,?,C,0,0,1,?,Mr.
42,43,0,3,"Kraeff, Mr. Theodor",male,?,0,0,349253,7.8958,?,C,0,0,1,?,Mr.


In [12]:
testdf[testdf['Age']=='?']

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilyCount,IsFemale,IsMale,Deck,Title
10,902,3,"Ilieff, Mr. Ylio",male,?,0,0,349220,7.8958,?,S,0,0,1,?,Mr.
22,914,1,"Flegenheim, Mrs. Alfred (Antoinette)",female,?,0,0,PC 17598,31.6833,?,S,0,1,0,?,Mrs.
29,921,3,"Samaan, Mr. Elias",male,?,2,0,2662,21.6792,?,C,2,0,1,?,Mr.
33,925,3,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",female,?,1,2,W./C. 6607,23.4500,?,S,3,1,0,?,Mrs.
36,928,3,"Roth, Miss. Sarah A",female,?,0,0,342712,8.0500,?,S,0,1,0,?,Miss.
39,931,3,"Hee, Mr. Ling",male,?,0,0,1601,56.4958,?,S,0,0,1,?,Mr.
41,933,1,"Franklin, Mr. Thomas Parham",male,?,0,0,113778,26.5500,D34,S,0,0,1,D,Mr.
47,939,3,"Shaughnessy, Mr. Patrick",male,?,0,0,370374,7.7500,?,Q,0,0,1,?,Mr.
54,946,2,"Mangiavacchi, Mr. Serafino Emilio",male,?,0,0,SC/A.3 2861,15.5792,?,C,0,0,1,?,Mr.
58,950,3,"Davison, Mr. Thomas Henry",male,?,1,0,386525,16.1000,?,S,1,0,1,?,Mr.


This is a judgement call: the rule of thumb is that you want to avoid imputing more than 25 percent of data, because then your final model is being fed too much bad data. I find that fare is useful in imputing this, but I'm going to scale it first.

In [13]:
ss = StandardScaler()
testdf.loc[152,'Fare']=13.6755
test_feature_engineered.loc[152,'Fare']=13.6755
titanic_scaled_features= ss.fit_transform(titanic_feature_engineered['Fare'])
test_scaled_features=ss.fit_transform(test_feature_engineered['Fare'])
    

/home/nerd/Data/Conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/nerd/Data/Conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/nerd/Data/Conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your 

In [14]:
testdf['Fare'].isnull().sum()

0

This means we have some missing data. let's take a look the missing value, and set it to be the average price for their passenger class. 

In [15]:
testdf[testdf['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilyCount,IsFemale,IsMale,Deck,Title


In [16]:
titanic[titanic['Pclass']==3]['Fare'].mean()

13.675550101832997

In [17]:
testdf.loc[152,'Fare']=13.6755
test_feature_engineered.loc[152,'Fare']=13.6755

In [18]:
testdf[testdf['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilyCount,IsFemale,IsMale,Deck,Title


In [19]:
titanic_scaled_features= ss.fit_transform(titanic_feature_engineered['Fare'])
test_scaled_features=ss.fit_transform(test_feature_engineered['Fare'])
    

/home/nerd/Data/Conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/nerd/Data/Conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/nerd/Data/Conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your 

In [20]:
titanic_feature_engineered['Fare']= titanic_scaled_features
test_feature_engineered['Fare']= test_scaled_features

In [21]:
trainagetrain =titanic_feature_engineered[(titanic_feature_engineered['Age']!='?')]
trainagetest = titanic_feature_engineered[(titanic_feature_engineered['Age']=='?')]
testagetest = test_feature_engineered[(test_feature_engineered['Age']=='?')]

In [22]:
#so, we have split our training data into data with ages and without, We've also grabbed our test data, because we're going to be imputing those missing ages as well

trainagetrain['Age']=trainagetrain.Age.apply(float)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
print(trainagetest.shape)
print(testagetest.shape)

(177, 72)
(86, 63)


In [24]:
# what follows is a quick and dirty imputation using a linear regression. I prefer this to the sklearn imputer function, but this takes more time. 
trainagetrain.corr()['Age'].sort_values()

Title_Master.   -0.398827
Pclass          -0.369226
plcass_3        -0.312271
SibSp           -0.308247
FamilyCount     -0.301914
parch_2         -0.278980
Title_Miss.     -0.276792
sibsp_4         -0.250853
Deck_?          -0.249732
Parch           -0.189119
family_3        -0.156019
parch_1         -0.155138
sibsp_3         -0.142148
family_5        -0.138675
family_6        -0.130889
sibsp_5         -0.112803
family_2        -0.097659
IsFemale        -0.093254
sibsp_2         -0.092893
family_7        -0.088989
Deck_F          -0.083970
Deck_G          -0.077296
Survived        -0.077221
family_4        -0.076528
Embarked__S     -0.032523
Embarked__Q     -0.022405
Title_Mlle.     -0.020808
Title_Mme.      -0.014703
Title_Ms.       -0.004384
plcass_2         0.006954
                   ...   
parch_6          0.034315
PassengerId      0.036847
Deck_T           0.039474
Embarked__C      0.046138
Title_Lady.      0.047214
Title_Sir.       0.049794
parch_5          0.054963
family_1    

In [25]:
agefeatures = ['parch_0','plcass_1','Title_Mr.', 'family_0', 'Title_Mrs.', 'Title_Master.', 'plcass_3', 'plcass_2', 'FamilyCount', 'SibSp' ,'Title_Miss.']

So, how I make a decission like this (quickly), is a look for high positive or negative correlations and try to reduce colinearity. 

In [26]:
age_X = trainagetrain[agefeatures] # this is how we are going to train the model on the training data for imputaiton
age_y = trainagetrain['Age'] #
age_train_X = trainagetest[agefeatures]
age_test_X = testagetest[agefeatures] # these are the features we are going to use to impute for the test set.

In [27]:
linreg = LinearRegression()
cross_val_score(linreg,age_X,age_y,cv=5)
# this is not great, but I think it's enough better than taking the mean or median that I'm going to move on. 


array([ 0.44826616,  0.33963644,  0.39240533,  0.3419173 ,  0.45179202])

In [28]:
titanic_feature_engineered.tail()

,Title_Capt.,Title_Col.,Title_Countess.,Title_Don.,Title_Dr.,Title_Jonkheer.,Title_Lady.,Title_Major.,Title_Master.,Title_Miss.,...,family_10,Deck_?,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
886,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
887,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
888,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
890,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [29]:
linreg.fit(age_X ,age_y)
age_train_X['Age']= linreg.predict(age_train_X)
age_test_X['Age']=linreg.predict(age_test_X)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Now let's sanity check our imputed ages. 


In [30]:
print(age_test_X.head()['Age'])
print(age_train_X.head()['Age'])

10    30.304688
22    47.281250
29    24.898438
33    29.445312
36    19.906250
Name: Age, dtype: float64
5     30.304688
17    34.210938
19    34.789062
26    30.304688
28    19.906250
Name: Age, dtype: float64


Now let's go ahead and write these values back to our dataframes. 


In [31]:
titanic_feature_engineered.loc[age_train_X.index,'Age'] = age_train_X['Age']
test_feature_engineered.loc[age_test_X.index,'Age'] = age_test_X['Age']


In [32]:
print(titanic_feature_engineered[titanic_feature_engineered['Age']=='?'])
print(test_feature_engineered[test_feature_engineered['Age']=='?'])
# Now we want to check  and make sure  that we don't have any missing ages in either dataframe. 


Empty DataFrame
Columns: [Title_Capt., Title_Col., Title_Countess., Title_Don., Title_Dr., Title_Jonkheer., Title_Lady., Title_Major., Title_Master., Title_Miss., Title_Mlle., Title_Mme., Title_Mr., Title_Mrs., Title_Ms., Title_Rev., Title_Sir., Embarked__C, Embarked__Q, Embarked__S, PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, FamilyCount, IsFemale, IsMale, Deck, Title, parch_0, parch_1, parch_2, parch_3, parch_4, parch_5, parch_6, plcass_1, plcass_2, plcass_3, sibsp_0, sibsp_1, sibsp_2, sibsp_3, sibsp_4, sibsp_5, sibsp_8, family_0, family_1, family_2, family_3, family_4, family_5, family_6, family_7, family_10, Deck_?, Deck_A, Deck_B, Deck_C, Deck_D, Deck_E, Deck_F, Deck_G, Deck_T]
Index: []

[0 rows x 72 columns]
Empty DataFrame
Columns: [Title_Col., Title_Dona., Title_Dr., Title_Master., Title_Miss., Title_Mr., Title_Mrs., Title_Ms., Title_Rev., Embarked__C, Embarked__Q, Embarked__S, PassengerId, Pclass, Name, Sex, Age, SibSp, Parch, T

In [33]:
titanic_feature_engineered.tail()

,Title_Capt.,Title_Col.,Title_Countess.,Title_Don.,Title_Dr.,Title_Jonkheer.,Title_Lady.,Title_Major.,Title_Master.,Title_Miss.,...,family_10,Deck_?,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
886,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
887,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
888,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
890,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


The empty dataframes mean that I've succesfully imputed ages. Now lets do age bands.  I want to do an age band for every thirteen years old, which I will later dummy. 



In [34]:
def ageband(age):
    ageband= (int(age)//13)+1
    return ageband
print(ageband(25))
print(ageband(26))
print(ageband(12))

2
3
1


In [35]:
titanic_feature_engineered['AgeBand']=titanic_feature_engineered.Age.apply(ageband)
test_feature_engineered['AgeBand']=test_feature_engineered.Age.apply(ageband)

Ther are a few names that jump out at me as having special meaning, so let's turn them into features. 

In [36]:
def isrev(name):
    if name == 'Rev.':
        return 1
    else:
        return 0
def isofficer(name):
    if (name == 'Col.') | (name=='Major.') | (name == 'Dr.'):
        return 1
    else:
        return 0
def isroyalty(name):
    if (name == 'Don.') | (name=='Jonkheer.') | (name == 'Sir.')| (name == 'Dona.')| (name == 'Lady.'):
        return 1
    else:
        return 0
    
   
titanic_feature_engineered['IsRev']=titanic_feature_engineered.Title.apply(isrev)
test_feature_engineered['IsRev']=test_feature_engineered.Title.apply(isrev)
titanic_feature_engineered['IsOfficer']=titanic_feature_engineered.Title.apply(isofficer)
test_feature_engineered['IsOfficer']=test_feature_engineered.Title.apply(isofficer)
titanic_feature_engineered['IsRoyalty']=titanic_feature_engineered.Title.apply(isroyalty)
test_feature_engineered['IsRoyalty']=test_feature_engineered.Title.apply(isroyalty)

     

As a note here, we're going to assign  mme as mrs, and mlle as miss, this will help us later with  some missing features. 

In [37]:
titanic_feature_engineered['Title'].value_counts()

Mr.          517
Miss.        182
Mrs.         125
Master.       40
Dr.            7
Rev.           6
Col.           2
Mlle.          2
Major.         2
Countess.      1
Jonkheer.      1
Mme.           1
Lady.          1
Capt.          1
Don.           1
Ms.            1
Sir.           1
Name: Title, dtype: int64

In [38]:
isroyalty('Jonkheer.')

1

In [39]:
titanic_feature_engineered.tail()

,Title_Capt.,Title_Col.,Title_Countess.,Title_Don.,Title_Dr.,Title_Jonkheer.,Title_Lady.,Title_Major.,Title_Master.,Title_Miss.,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,AgeBand,IsRev,IsOfficer,IsRoyalty
886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,1,0,0
887,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,2,0,0,0
888,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,2,0,0,0
889,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,3,0,0,0
890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0


Now it's time for some wonky feature engineering: 
I've discovered that from people's names, you can find if they have a dead husband or a dead wife. (because married women are listed as Mrs (Husband). Here's some feature extraction to find all of the dead and living men and women in the training set, and to add a variable for dead vs living spouse. 
This probably can't be generalized to many other data-sets, but it was fun "insight" into the name. 


In [40]:
def findsurname(wife):
    if wife[-1]==')':
        return wife.split()[0]
    
def findhusband(wife, husband):
    for num in range(len(wife.split())):
        if wife.split()[num][-1]=='.':
            try:
                return (wife.split()[num-1]==husband.split()[num-1])  & (wife.split()[num+1]==husband.split()[num+1])
            except:
                return False
            
        #Splitting by gender
def dead_husband(wife,husbandlist):
    #this takes a wife's name as an argument and a listas the second argument.  it returns a true or false
    result= False # assume that the wife did not die
    for num in range(len(husbandlist)):
        if findhusband(wife, husbandlist[num]):
            result = True
    return result
def dead_wife(husband,wifelist):
    result= False # assume that the wife did not die
    for num in range(len(wifelist)):
        if findhusband(wifelist[num], husband):
            result = True
    return result
            
titanicmale=  titanic[titanic['Sex']=='male'] 
titanicfemale= titanic[titanic['Sex']=='female']
testmale=  testdf[testdf['Sex']=='male'] 
testfemale= testdf[testdf['Sex']=='female']



In [41]:
#it's important to note that this is a pretty expensive function to build for theses features if I use the dataframe. I'm iterating over the entire data frame for every row in the dataframe. lets see if I can make it better. 
# let's make a list of dead women and dead men.  Lists return faster. 
livemen= titanicmale[titanicmale['Survived']==1]
deadmen= titanicmale[titanicmale['Survived']==0]
livewomen= titanicfemale[titanicfemale['Survived']==1]
deadwomen= titanicfemale[titanicfemale['Survived']==0]
deadwomennames= deadwomen.Name.values
livewomennames= livewomen.Name.values
livemennames = livemen.Name.values
deadmennames = deadmen.Name.values

In [42]:
# lets do women with dead husbands. 
titanicfemale['Dead_Husband']= titanicfemale.Name.apply(lambda x:  dead_husband(x,deadmennames))
titanicfemale['Live_Husband']= titanicfemale.Name.apply(lambda x:  dead_husband(x,livemennames))
titanicmale['Dead_Wife']=titanicmale.Name.apply(lambda x:  dead_wife(x,deadwomennames))
titanicmale['Live_Wife']=titanicmale.Name.apply(lambda x:  dead_wife(x,livewomennames))
# lets do women with dead husbands. 
testfemale['Dead_Husband']= testfemale.Name.apply(lambda x:  dead_husband(x,deadmennames))
testfemale['Live_Husband']= testfemale.Name.apply(lambda x:  dead_husband(x,livemennames))
testmale['Dead_Wife']=testmale.Name.apply(lambda x:  dead_wife(x,deadwomennames))
testmale['Live_Wife']=testmale.Name.apply(lambda x:  dead_wife(x,livewomennames))


/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [43]:
test_feature_engineered.tail()

,Title_Col.,Title_Dona.,Title_Dr.,Title_Master.,Title_Miss.,Title_Mr.,Title_Mrs.,Title_Ms.,Title_Rev.,Embarked__C,...,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,AgeBand,IsRev,IsOfficer,IsRoyalty
413,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
414,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,4,0,0,1
415,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
416,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
417,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [44]:


test_feature_engineered.loc[testfemale.index,'Dead_Husband']= testfemale['Dead_Husband'].astype(int)

test_feature_engineered.loc[testfemale.index,'Live_Husband']= testfemale['Live_Husband'].astype(int)
test_feature_engineered.loc[testmale.index,'Dead_Wife']= testmale['Dead_Wife'].astype(int)
test_feature_engineered.loc[testmale.index,'Live_Wife']= testmale['Live_Wife'].astype(int)
titanic_feature_engineered.loc[titanicfemale.index,'Dead_Husband']= titanicfemale['Dead_Husband'].astype(int)
titanic_feature_engineered.loc[titanicfemale.index,'Live_Husband']= titanicfemale['Live_Husband'].astype(int)
titanic_feature_engineered.loc[titanicmale.index,'Dead_Wife']= titanicmale['Dead_Wife'].astype(int)
titanic_feature_engineered.loc[titanicmale.index,'Live_Wife']= titanicmale['Live_Wife'].astype(int)


In [45]:
titanic_feature_engineered[titanic_feature_engineered['Live_Wife']==1]

,Title_Capt.,Title_Col.,Title_Countess.,Title_Don.,Title_Dr.,Title_Jonkheer.,Title_Lady.,Title_Major.,Title_Master.,Title_Miss.,...,Deck_G,Deck_T,AgeBand,IsRev,IsOfficer,IsRoyalty,Dead_Husband,Live_Husband,Dead_Wife,Live_Wife
34,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,NaN,NaN,0.0,1.0
35,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,NaN,NaN,0.0,1.0
62,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,NaN,NaN,0.0,1.0
99,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,NaN,NaN,0.0,1.0
122,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,NaN,NaN,0.0,1.0
137,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,NaN,NaN,0.0,1.0
165,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,NaN,NaN,0.0,1.0
206,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,NaN,NaN,0.0,1.0
217,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,NaN,NaN,0.0,1.0
224,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,NaN,NaN,0.0,1.0


In [46]:

age_dummies= pd.get_dummies(titanic_feature_engineered.AgeBand, prefix='AgeBand_')
titanic_feature_engineered = pd.concat([titanic_feature_engineered,age_dummies],axis=1)
titanic_feature_engineered.head()['AgeBand']
test_age_dummies= pd.get_dummies(titanic_feature_engineered.AgeBand, prefix='AgeBand_')
test_feature_engineered = pd.concat([test_feature_engineered,test_age_dummies],axis=1)




In [47]:
titanic_feature_engineered.fillna(value=0, inplace=True)
test_feature_engineered.fillna(value=0, inplace=True)

In [48]:
testdf.shape

(418, 16)

In [49]:
test_feature_engineered.shape

(891, 78)

Now that we've finished feature engineering let's look at our correlations and decide on a model. 

In [50]:

titanic_feature_engineered.corr()['Survived'].sort_values()

Title_Mr.         -0.549199
IsMale            -0.543351
Pclass            -0.338481
plcass_3          -0.322308
Deck_?            -0.316912
family_0          -0.203367
Embarked__S       -0.155660
parch_0           -0.147408
sibsp_0           -0.115867
AgeBand__6        -0.084089
family_5          -0.080968
Age               -0.072567
sibsp_8           -0.070234
family_10         -0.070234
Dead_Wife         -0.067344
family_7          -0.064988
Title_Rev.        -0.064988
IsRev             -0.064988
sibsp_4           -0.064123
AgeBand           -0.063817
AgeBand__3        -0.059351
sibsp_5           -0.059292
parch_4           -0.053002
family_4          -0.049466
sibsp_3           -0.037215
SibSp             -0.035322
parch_5           -0.028398
Title_Jonkheer.   -0.026456
Title_Capt.       -0.026456
parch_6           -0.026456
                     ...   
AgeBand__7         0.042470
Title_Sir.         0.042470
Title_Lady.        0.042470
Title_Ms.          0.042470
Title_Mme.         0

In [51]:
possiblefeatures= titanic_feature_engineered.corr()['Survived'].sort_values().index

As a way of sorting this, I use the output of this correlation matrix to choose features. Once Again, I'm mainly looking for high negative or postive correlations that aren't related to other features. 
As a note, it looks like my hypothesis was correct, a males wife having died is a useful indicator that he is also going to die. 

In [52]:
possiblefeaturelist = [possiblefeatures[1],possiblefeatures[3],possiblefeatures[4],possiblefeatures[5],possiblefeatures[6],possiblefeatures[7],
           possiblefeatures[0],possiblefeatures[-21],possiblefeatures[-4],possiblefeatures[-5],possiblefeatures[-6],possiblefeatures[-7], 
                      possiblefeatures[-8],possiblefeatures[-9],possiblefeatures[-18],possiblefeatures[8],possiblefeatures[9],possiblefeatures[10],possiblefeatures[11],
                      possiblefeatures[-17],possiblefeatures[-16],possiblefeatures[-15],possiblefeatures[-14],possiblefeatures[-13],possiblefeatures[-12],possiblefeatures[-11],possiblefeatures[-19],possiblefeatures[-20]]         

possiblefeaturelist

['IsMale',
 'plcass_3',
 'Deck_?',
 'family_0',
 'Embarked__S',
 'parch_0',
 'Title_Mr.',
 'Title_Master.',
 'Title_Miss.',
 'plcass_1',
 'Fare',
 'Deck_B',
 'Embarked__C',
 'sibsp_1',
 'Deck_C',
 'sibsp_0',
 'AgeBand__6',
 'family_5',
 'Age',
 'family_3',
 'parch_1',
 'Dead_Husband',
 'Live_Husband',
 'family_2',
 'Deck_E',
 'Deck_D',
 'AgeBand__1',
 'plcass_2']

In [100]:
features = [ 'Deck_?', 'Deck_B','Deck_C','Deck_D','Deck_E', 'IsFemale',
 'plcass_3','plcass_1','plcass_2',
 'family_0','family_1','family_3','family_5','family_7', 'family_10',
 'Embarked__S','Embarked__C',
 'Fare','AgeBand',
 'AgeBand__1','AgeBand__6',
'Dead_Husband','Live_Wife','Live_Husband','Dead_Wife', 'Title_Mr.']

In [91]:
y= titanic_feature_engineered['Survived']
X= titanic_feature_engineered[features]
logreg = LogisticRegression(fit_intercept = False)
cross_val_score(logreg, X,y,cv =10, scoring='accuracy').mean()

0.79914652139371234

In [101]:
polyfit = PolynomialFeatures()

In [106]:
X_poly = polyfit.fit_transform(X)
X_poly_scaled= ss.fit_transform(X_poly)

In [108]:
cross_val_score(logreg, X_poly,y,cv =10, scoring='accuracy').mean()

0.81153132448076271

In [92]:
X_scaled = ss.fit_transform(X)
cross_val_score(logreg, X_scaled,y,cv =10, scoring='accuracy').mean()

0.79237969583475198

This tells me I can expect 80 percentish accuracy with these features. It doesn't look like scaling any feature except for ticket price (which we scaled much earlier), helps us. 
Let's go ahead and grid search and find the best logistic regression. 


In [93]:
Crange=np.linspace(1.5,3)
logreg_params= {'C':Crange,'penalty':['l1','l2']}

gs2= GridSearchCV(logreg, logreg_params,cv=10)
#print(Crange)

In [94]:
gs2.fit(X,y)
print(gs2.best_score_)
print(gs2.best_params_)

0.806958473625
{'C': 3.0, 'penalty': 'l1'}


In [58]:
#test_feature_engineered.columns

In [96]:
model = LogisticRegression(C=gs2.best_params_['C'], penalty=gs2.best_params_['penalty'])
X_test= test_feature_engineered[features]
model.fit(X, y)
#ss.fit(X)
#X_test_scaled= ss.transform(X_test)
pred= model.predict(X_test)
test_feature_engineered['PassengerId']= test_feature_engineered.PassengerId.apply(int)
submission1  = pd.DataFrame({'Survived':pred}, index=test_feature_engineered.PassengerId.values)
submission1.drop(0,inplace =True)
submission1.to_csv('submission11.csv', index_label='PassengerId')
submission1.shape

(418, 1)

In [60]:
testdf.shape

(418, 16)

In [61]:
pred

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0,

Let's compare this to k nearest neighbors

In [97]:
knn =KNeighborsClassifier( n_jobs=-1)
cross_val_score(knn, X,y,cv =10, scoring='accuracy').mean()

0.7936786403359436

In [109]:
cross_val_score(knn, X_poly,y,cv =10, scoring='accuracy').mean()

0.7856486210418796

In [98]:
cross_val_score(knn, X_scaled,y,cv =10, scoring='accuracy').mean()

0.78475314947225061

In [110]:
cross_val_score(knn, X_poly_scaled,y,cv =10, scoring='accuracy').mean()

0.79260640108954716

In [64]:
nrange=list(range(1,45))
knn_params= {'n_neighbors':nrange,'weights':['uniform','distance'],'p':[1,1.5,1.25,1.75,2]}

gs= GridSearchCV(knn, knn_params,cv=10)

It's important to note that the grid search below can take a long time.  (I ran this on a system using a gtx 1060 with hw accelerated numpy/sklearn, but I could have done this on any computer provided I was willing to wait longer.)

In [111]:
X_scaled= ss.fit_transform(X)
gs.fit(X_poly_scaled,y)
print(gs.best_score_)
print(gs.best_params_)

0.810325476992
{'n_neighbors': 20, 'p': 1, 'weights': 'uniform'}


In [73]:
model = KNeighborsClassifier(n_jobs=-1, n_neighbors=gs.best_params_['n_neighbors'], p=gs.best_params['p'], weights=gs.best_params['weights'])
X_test= test_feature_engineered[features]
model.fit(X_scaled, y)
ss.fit(X)
X_test_scaled= ss.transform(X_test)

pred= model.predict(X_test_scaled)
test_feature_engineered['PassengerId']= test_feature_engineered.PassengerId.apply(int)
submission2  = pd.DataFrame({'Survived':pred}, index=test_feature_engineered.PassengerId.values)
submission2.drop(0,inplace =True)
submission2.to_csv('submission22.csv', index_label='PassengerId')


In [67]:
#submission.drop(0,inplace =True)

In [68]:
svm1 = svm.SVC(kernel='linear')
svm2 = svm.SVC(kernel='poly')
svm3= svm.SVC(kernel='rbf')

In [69]:
print(cross_val_score(svm1, X,y,cv =5, scoring='accuracy').mean())
print(cross_val_score(svm2, X,y,cv =5, scoring='accuracy').mean())
print(cross_val_score(svm3, X,y,cv =5, scoring='accuracy').mean())

0.802489727061
0.630778282189
0.79793229063


In [70]:
# now we can compare all features scaled to just the ones that I want scaled. 

print(cross_val_score(svm1, X_scaled,y,cv =5, scoring='accuracy').mean())
print(cross_val_score(svm2, X_scaled,y,cv =5, scoring='accuracy').mean())
print(cross_val_score(svm3, X_scaled,y,cv =5, scoring='accuracy').mean())

0.802489727061
0.80928187129
0.817115583553


In [71]:
Crange=np.linspace(.5,1)
#Crange= [0.1,.5,.75,.78,.79,.74,.70,.65]
svm_params= {'C':Crange}

gs3= GridSearchCV(svm3, svm_params,cv=10)
#print(Crange)

In [74]:
gs3.fit(X_scaled,y)
print(gs3.best_score_)
print(gs3.best_params_)

0.821548821549
{'C': 0.70408163265306123}


In [ ]:
print(gs3.best_score_)
print(gs3.best_params_)

So what I  do with these three trained models is output a .csv, and upload them to kaggle. 
csv1 will be log, csv2 will be knn, csv3 will be svm.Another way we could try to improve accuracy is ensemble these multiples models together( or stack them) so that, for example we use a simple majority of the three models to determine our prediction.  rather than using a function for this, we will stack manually and call this csv 4. 


In [75]:
model =  svm.SVC(kernel='rbf', C=.70 )
X_test= test_feature_engineered[features]
model.fit(X_scaled, y)
ss.fit(X)
X_test_scaled= ss.transform(X_test)
pred= model.predict(X_test_scaled)
submission3  = pd.DataFrame({'Survived':pred}, index=test_feature_engineered.PassengerId.values )
submission3.drop(0,inplace =True)
submission3.to_csv('submission33.csv',index_label='PassengerId')

In [76]:
submission1.head()

,Survived
892,0
893,1
894,0
895,0
896,1


In [77]:
submission2.head()

,Survived
892,0
893,1
894,0
895,0
896,0


In [78]:
submission3.head()

,Survived
892,0
893,1
894,0
895,0
896,1


In [79]:
submission1['survived1']=submission1['Survived']
submission2['survived2']=submission2['Survived']
submission3['survived3']=submission3['Survived']

In [80]:
submissions = pd.concat([submission1,submission2,submission3], axis=1)

In [81]:
submissions.head()

,Survived,survived1,Survived,survived2,Survived,survived3
892,0,0,0,0,0,0
893,1,1,1,1,1,1
894,0,0,0,0,0,0
895,0,0,0,0,0,0
896,1,1,0,0,1,1


In [82]:
submissions['total']=submissions['survived1']+submissions['survived2']+submissions['survived3']

In [83]:
submissions.drop('Survived', axis=1, inplace=True)

In [84]:
submissions['Survived'] = submissions['total']//2

In [85]:
submissions.head()

,survived1,survived2,survived3,total,Survived
892,0,0,0,0,0
893,1,1,1,3,1
894,0,0,0,0,0
895,0,0,0,0,0
896,1,0,1,2,1


In [86]:
submissions.drop('total',axis=1,inplace=True)
submissions.drop('survived1',axis=1,inplace=True)
submissions.drop('survived2',axis=1,inplace=True)
submissions.drop('survived3',axis=1,inplace=True)

In [ ]:
submissions.tail()

In [1]:
submissions.to_csv('submission4.csv',index_label='PassengerId')

NameError: name 'submissions' is not defined